## Check Equilibrium
This notebook reads files describing a structure, and the files output by Frame2D after an
analysis, and checks that the forces and moments on every node are in equilibrium.

It does this in the simplest way possible, using quite different logic than Frame2D, resulting
in a higher degree of confidence in the results.  It would have been better had someone else
programmed it, but oh well ...

In [1]:
ds = 'KG82'
lcase = 'all'
ds = 'l22x6'
lcase = 'Case-2b'

def filename(basename,lc=None):
    if lc is not None:
        basename = lc + '/' + basename
    return 'data/' + ds + '.d/' + basename + '.csv'

def Warn(msg):
    print('!!!!! Warning: {}'.format(msg))

In [2]:
import pandas as pd
import math

In [3]:
class Node(object):
    
    def __init__(self,id,x,y):
        self.id = id
        self.x = x
        self.y = y
        self.sumFX = 0.
        self.sumFY = 0.
        self.sumMZ = 0.

In [4]:
table = pd.read_csv(filename('nodes'))
NODES = {}
for i,n in table.iterrows():
    if n.NODEID in NODES:
        Warn("Node '{}' is multiply defined.".format(n.NODEID))
    NODES[n.NODEID] = Node(n.NODEID,float(n.X),float(n.Y))

In [5]:
class Member(object):
    
    def __init__(self,id,nodej,nodek):
        self.id = id
        self.nodej = nodej
        self.nodek = nodek
        
        dx = nodek.x - nodej.x
        dy = nodek.y - nodej.y
        self.L = L = math.sqrt(dx*dx + dy*dy)
        self.cosx = dx/L
        self.cosy = dy/L

In [6]:
table = pd.read_csv(filename('members'))
MEMBERS = {}
for i,m in table.iterrows():
    if m.MEMBERID in MEMBERS:
        Warn("Member '{}' is multiply defined.".format(m.MEMBERID))
    MEMBERS[m.MEMBERID] = Member(m.MEMBERID,NODES[m.NODEJ],NODES[m.NODEK])

In [7]:
try:
    lctable = pd.read_csv(filename('load_combinations'))
    use_all = False
    COMBO = {}
    for i,row in lctable.iterrows():
        if row.CASE == lcase:
            COMBO[row.LOAD.lower()] = row.FACTOR
except OSError:
    use_all = True
    COMBO = None
COMBO

{'dead': 1.25, 'live': 1.5, 'wind': 0.4}

In [8]:
table = pd.read_csv(filename('node_loads'))
for i,p in table.iterrows():
    f = 1.0 if use_all else COMBO.get(p.LOAD.lower(),0.)
    dirn = p.DIRN.upper()
    if dirn in ['FX','FY','MZ']:
        n = NODES[p.NODEID]
        a = 'sum'+dirn
        setattr(n,a,getattr(n,a,0.)+float(p.F)*f)
    else:
        Warn("Direction '{}' invalid for node '{}'.".format(dirn,p.NODEID))

In [9]:
try:
    table = pd.read_csv(filename('pdelta_forces'))
    for i,p in table.iterrows():
        dirn = p.DIRN.upper()
        if dirn in ['FX','FY','MZ']:
            n = NODES[p.ID]
            a = 'sum'+dirn
            setattr(n,a,getattr(n,a,0.)+float(p.F))
        else:
            Warn("Direction '{}' invalid for node '{}'.".format(dirn,p.ID))
except OSError:
    pass

In [10]:
table = pd.read_csv(filename('reaction_forces',lcase))
for i,r in table.iterrows():
    n = NODES[r.NODEID]
    n.sumFX += 0. if pd.isnull(r.FX) else float(r.FX)
    n.sumFY += 0. if pd.isnull(r.FY) else float(r.FY)
    n.sumMZ += 0. if pd.isnull(r.MZ) else float(r.MZ)
table

,NODEID,FX,FY,MZ
0,A0,-114985.838246,8191415.004852,NaN
1,B0,-150393.851305,14822984.995148,NaN
2,C0,-120336.218178,12992089.267568,NaN
3,D0,-154495.099476,15460977.543879,NaN
4,E0,-144093.356820,16090933.188554,NaN
5,F0,-121127.687706,11963883.967525,NaN
6,G0,-156567.948266,11050516.032475,NaN


In [11]:
mtable = pd.read_csv(filename('member_end_forces',lcase))
for i,row in mtable.iterrows():
    m = MEMBERS[row.MEMBERID]
    n = m.nodej
    n.sumFX -= row.FXJ*m.cosx - row.FYJ*m.cosy
    n.sumFY -= row.FXJ*m.cosy + row.FYJ*m.cosx
    n.sumMZ -= row.MZJ
    n = m.nodek
    n.sumFX -= row.FXK*m.cosx - row.FYK*m.cosy
    n.sumFY -= row.FXK*m.cosy + row.FYK*m.cosx
    n.sumMZ -= row.MZK
mtable

,MEMBERID,FXJ,FYJ,MZJ,FXK,FYK,MZK
0,CA0A1,8191415.004852,114985.838246,-6.832124e-08,-8191415.004852,-114985.838246,7.474079e+08
1,CB0B1,14822984.995148,150393.851305,-8.134521e-08,-14822984.995148,-150393.851305,9.775600e+08
2,CC0C1,12992089.267568,120336.218178,2.067682e-07,-12992089.267568,-120336.218178,7.821854e+08
3,CD0D1,15460977.543879,154495.099476,-1.721673e-07,-15460977.543879,-154495.099476,1.004218e+09
4,CE0E1,16090933.188554,144093.356820,-1.208828e-07,-16090933.188554,-144093.356820,9.366068e+08
5,CF0F1,11963883.967525,121127.687706,-4.719186e-08,-11963883.967525,-121127.687706,7.873300e+08
6,CG0G1,11050516.032475,156567.948266,-7.818016e-09,-11050516.032475,-156567.948266,1.017692e+09
7,CA1A2,8045138.190965,17954.020340,1.989471e+07,-8045138.190965,-17954.020340,7.885241e+07
8,CB1B2,13902861.809035,188577.930403,5.118308e+08,-13902861.809035,-188577.930403,5.253479e+08
9,CC1C2,12541665.630852,51577.397186,1.062672e+08,-12541665.630852,-51577.397186,1.774084e+08


In [12]:
maxF = max([mtable[c].abs().max() for c in 'FXJ FXK FYJ FYK'.split()])
maxM = max([mtable[c].abs().max() for c in 'MZJ MZK'.split()])
maxF,maxM

(16090933.188553501, 1624390432.9707)

In [13]:
sums = pd.DataFrame([(n.id,n.sumFX,n.sumFY,n.sumMZ) for n in NODES.values()],
                    columns=['ID','sumFX','sumFY','sumMZ']).set_index(['ID'])
sums

lm = 1E-11
sums['sumFX'][sums['sumFX'].abs() <= maxF*lm] = 0
sums['sumFY'][sums['sumFY'].abs() <= maxF*lm] = 0
sums['sumMZ'][sums['sumMZ'].abs() <= maxM*lm] = 0
sums

,sumFX,sumFY,sumMZ
ID,,,
F6,0,0,0
A20,0,0,0
A22,0,0,0
A1,0,0,0
E3,0,0,0
D11,0,0,0
B8,0,0,0
D12,0,0,0
F8,0,0,0


In [14]:
sums.abs().max()

sumFX    0
sumFY    0
sumMZ    0
dtype: float64